In [26]:
import geopandas as gpd
from folium import Choropleth, Circle, Marker
from folium.plugins import MarkerCluster, HeatMap
import folium
import pandas as pd
import math

We define a function `embed_map()` for displaying interactive maps. It accepts two arguments: the variable containing the map, and the name of the HTML file where the map will be saved.

This function ensures that the maps are visible [in all web browsers](https://github.com/python-visualization/folium/issues/812).

In [2]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

Map of boston 

In [8]:
m1 = folium.Map(location=[42.32,-71.0589], tiles ='openstreetmap', zoom_start=10)
m1

Now, we'll add some crime data to the map!

We won't focus on the data loading step. Instead, you can imagine you are at a point where you already have the data in a pandas DataFrame crimes. The first five rows of the data are shown below.

In [18]:
crimes = pd.read_csv('Boston_crimes.csv', encoding ='latin1')
crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


NOTE: Loading the Boston file without `encoding='latin1'` raises a `UnicodeDecoderError`: 'utf-8 codec can't decode byte 0xa0 in position 24: Invalid start byte. 

solving this requires setting encoding ='latin1'

Plotting points

To reduce the amount of data we need to fit on the map, we'll (temporarily) confine our attention to daytime robberies.

In [21]:
dayrob = crimes[((crimes['OFFENSE_CODE_GROUP']=='Robbery') & (crimes['HOUR'].isin(range(6, 19))))]
dayrob.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
42,I182070888,301,Robbery,ROBBERY - STREET,A1,76,NaN,2018-09-03 15:45:00,2018,9,Monday,15,Part One,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
62,I182070865,301,Robbery,ROBBERY - STREET,NaN,,NaN,2018-09-03 15:54:00,2018,9,Monday,15,Part One,NaN,42.315809,-71.098290,"(42.31580852, -71.09829029)"
273,I182070621,361,Robbery,ROBBERY - OTHER,A7,23,NaN,2018-09-02 18:09:00,2018,9,Sunday,18,Part One,MARION ST,42.377256,-71.036674,"(42.37725644, -71.03667354)"
299,I182070598,311,Robbery,ROBBERY - COMMERCIAL,A15,60,NaN,2018-09-02 17:15:00,2018,9,Sunday,17,Part One,RUTHERFORD AVE,42.371673,-71.063264,"(42.37167264, -71.06326413)"
345,I182070549,301,Robbery,ROBBERY - STREET,B2,282,NaN,2018-09-02 07:00:00,2018,9,Sunday,7,Part One,WARREN ST,42.328234,-71.083290,"(42.32823419, -71.08328981)"


Now we want to add markers to the map with folium.Marker(). Each marker below corresponds to a different robbery.

We've already imported Marker() from folium we will just go ahead and use it

In [27]:
m2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

#Add points were crimes were committed to map
for idx, row in dayrob.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        Marker([row['Lat'], row['Long']]).add_to(m2)
m2
    

NOTE: Location values can't contain NaN (Not a Number). So we use math.isnan to make sure only rows with real values are used.

folium.plugins.MarkerCluster

If we have a lot of markers to add, folium.plugins.MarkerCluster() can help to declutter the map. Each marker is added to a MarkerCluster object. Decluttering means that the closest points will be grouped under one main cluster.The individual points will be seen if we zoom in to that particular 'ball'. This ensures that our map doesn't look too stuffy.

In [40]:
m3 = folium.Map(location=[42.32,-71.0589], tiles ='cartodbpositron', zoom_start=12)

#Add MarkerCluster 
mc=MarkerCluster()
for idx, row in dayrob.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']]))
m3.add_child(mc)
m3